In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load dos dados do ficheiro JSON e preparação dos dados para treino e teste
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    captured_data = data['capturedData']

    accelerometerXAxis = [entry['accelerometerXAxis'] for entry in captured_data]
    accelerometerYAxis = [entry['accelerometerYAxis'] for entry in captured_data]
    accelerometerZAxis = [entry['accelerometerZAxis'] for entry in captured_data]
    gyroscopeXAxis = [entry['gyroscopeXAxis'] for entry in captured_data]
    gyroscopeYAxis = [entry['gyroscopeYAxis'] for entry in captured_data]
    gyroscopeZAxis = [entry['gyroscopeZAxis'] for entry in captured_data]

    # criar dataFrame 
    df = pd.DataFrame({
        'accelerometerXAxis': accelerometerXAxis,
        'accelerometerYAxis': accelerometerYAxis,
        'accelerometerZAxis': accelerometerZAxis,
        'gyroscopeXAxis': gyroscopeXAxis,
        'gyroscopeYAxis': gyroscopeYAxis,
        'gyroscopeZAxis': gyroscopeZAxis
    })
    
    print("Dataframe:")
    print(df)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df.values)

    sequence_length = 11
    sequences = []
    for i in range(len(scaled_data) - sequence_length):
        sequences.append(scaled_data[i:i+sequence_length])

    sequences = np.array(sequences)

    # conjuntos de treino e teste
    X = sequences[:, :-1]
    y = sequences[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, scaler

#treinar o modelo LSTM
def build_and_train_model(X_train, X_test, y_train, y_test):
    # construção do LSTM
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(X_train.shape[2])) 
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

    return model

# prever os valores e mostrar a comparação
def predict_and_compare(model, X_test, y_test, scaler):
    y_pred = model.predict(X_test)

    y_pred_inverse = scaler.inverse_transform(y_pred)
    y_test_inverse = scaler.inverse_transform(y_test)

    comparison_df = pd.DataFrame({'Real': y_test_inverse.flatten(), 'Predicted': y_pred_inverse.flatten()})

    real_values = comparison_df['Real']
    predicted_values = comparison_df['Predicted']
    percent_difference = ((predicted_values - real_values) / real_values) * 100

    comparison_df['Percent Difference (%)'] = percent_difference

    print(comparison_df)

# menu
def main():
    datasets = [
        # BMW e36 datasets 10m
        r'..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\normal\BMW e36\acceleration 0m 10m normal-23-04-2023-19-30-17.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\slow\BMW e36\acceleration 0m 10m slow-23-04-2023-19-19-20.json',

    ]

    print("Dataset:")
    for i, dataset in enumerate(datasets):
        print(f"{i+1}. {dataset}")

    choice = int(input("Escolha o dataset a usar: ")) - 1
    chosen_dataset = datasets[choice]
    print("\n\nDataset escolhido:", chosen_dataset)

    X_train, X_test, y_train, y_test, scaler = load_dataset(datasets[choice])
    model = build_and_train_model(X_train, X_test, y_train, y_test)
    predict_and_compare(model, X_test, y_test, scaler)

if __name__ == "__main__":
    main()



Dataset:
1. ..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json
2. ..\Datasets\Acceleration\Till reached distance\10m\normal\BMW e36\acceleration 0m 10m normal-23-04-2023-19-30-17.json
3. ..\Datasets\Acceleration\Till reached distance\10m\slow\BMW e36\acceleration 0m 10m slow-23-04-2023-19-19-20.json


Dataset escolhido: ..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json
Dataframe:
    accelerometerXAxis  accelerometerYAxis  accelerometerZAxis  \
0             0.204224            0.574472            0.015077   
1             0.230342            0.175526            0.122981   
2             0.199742            1.303585           -0.188207   
3             0.003083            0.753614            0.103433   
4             0.202767            0.281722            0.290354   
5            -0.042388           -0.066736           -0.030522   
6  